In [2]:
import talib as ta
import numpy as np
import pandas as pd

In [14]:
ASSET_DETAILS_CSV = './data/asset_details.csv'
df_asset_details = pd.read_csv(ASSET_DETAILS_CSV).sort_values("Asset_ID")
add_weight_map = dict(zip(df_asset_details.Asset_ID, 
                        df_asset_details.Weight/df_asset_details.Weight.sum()))

sup_train = pd.read_csv('./data/supplemental_train.csv')
sup_train['Weight'] = sup_train['Asset_ID'].map(add_weight_map)
sup_train = sup_train.sort_values('timestamp').set_index(["timestamp",'Asset_ID'])
sup_train.drop('Target',axis=1, inplace=True)


def log_return(series, periods=5):
    return np.log(series).diff(periods)
lr_15 = sup_train.groupby('Asset_ID').apply( 
        lambda x: log_return(x[['Close']],15)
        )
sup_train['lr_15'] = lr_15['Close']

mkt_lr_15 = sup_train.groupby('timestamp').apply( 
    lambda x: x[["lr_15", "Close"]].multiply(x["Weight"], axis="index").sum(skipna=True)
    )
mkt_lr_15.columns = ['Mkt_lrt_15','Crypto_Index']
firsts = sup_train.index.get_level_values('timestamp')
sup_train[['Mkt_lrt_15','Crypto_Index']] = mkt_lr_15.loc[firsts].values

In [17]:
sup_train.index.get_level_values('timestamp').min(),sup_train.index.get_level_values('timestamp').max()

(1632182460, 1641772800)

In [25]:
sup_train.shape

(2236496, 11)

In [90]:
def lag_features(df, fastk1,fastk2,adx,macd_s,macd_l,macd_sig,rsi,vol_sum,std_Crypto_Index,std_lr_15,std_Mkt_lrt_15, **kwargs):    
    ####TECH indicators
    df['slowK'], df['slowD'] = ta.STOCH(df.High, df.Low, df.Close, 
                                        fastk_period=fastk1, slowk_period=int(3*fastk1/5), slowd_period=int(3*fastk1/5),
                                        slowk_matype=0, slowd_matype=0)
    df['fastK'], df['fastD'] = ta.STOCHF(df.High, df.Low, df.Close,
                                         fastk_period=fastk2, fastd_period=int(3*fastk2/5), 
                                         fastd_matype=0)
    df[f'rsi_{rsi}'] = ta.RSI(df['Close'], timeperiod=rsi)
    df[f'macd_{macd_s}_{macd_l}'],df[f'macd_signal_{macd_sig}'], df['macd_hist'] = \
                ta.MACD(df['Close'],fastperiod=macd_s, slowperiod=macd_l, signalperiod=macd_sig)
    df[f'adx_{adx}'] = ta.ADX(df['High'], df['Low'],df['Close'], timeperiod=adx)#Average Directional Movement Index
    #df['AD'] = ta.AD(df['High'], df['Low'],df['Close'], df['Volume'])#Accumulation Distribution Line
    df[f'vol_sum_{vol_sum}'] = ta.SMA(df['Volume'],vol_sum)*vol_sum
    ####std volatility
    df[f'std_lr_15_{std_lr_15}'] = ta.STDDEV(df.lr_15,timeperiod=std_lr_15, nbdev=1)
    df[f'std_Mkt_lrt_15_{std_Mkt_lrt_15}'] = ta.STDDEV(df.Mkt_lrt_15,timeperiod=std_Mkt_lrt_15, nbdev=1)
    df[f'std_Crypto_Index_{std_Crypto_Index}'] = ta.STDDEV(df.Crypto_Index,timeperiod=std_Crypto_Index, nbdev=1)
    return df



In [91]:
fdict={'std_lr_15': 5,
 'std_Mkt_lrt_15': 10,
 'std_Crypto_Index': 15,
 'rsi': 60,
 'vol_sum':15,
 'macd_sig': 5,
 'macd_s': 15,
 'macd_l': 25,
 'lrtn': 50,
 'fastk2': 15,
 'fastk1': 5,
 'beta_s': '6h',
 'beta_l': '2d',
 'adx': 40}

pre_minute = 2000
sup_train2 = sup_train.iloc[-14*pre_minute:,:]
sup_train2=sup_train2.groupby('Asset_ID').apply(lambda x: lag_features(x,**fdict))
nan_num = []
nan_mins = []
for col in sup_train2.columns:
    nan_num.append(sup_train2[col].isin([np.nan]).sum())
    nan_mins.append(nan_num[-1]/14)
    print(f"{col}: {nan_num[-1]} rows, {nan_mins[-1]} mins")
print(f"max nan mins={max(nan_mins)}")

sup_train2[sup_train2.isin([np.nan]).any(axis=1)]
sup_train2.iloc[-14:,8:]

Count: 0 rows, 0.0 mins
Open: 0 rows, 0.0 mins
High: 0 rows, 0.0 mins
Low: 0 rows, 0.0 mins
Close: 0 rows, 0.0 mins
Volume: 0 rows, 0.0 mins
VWAP: 0 rows, 0.0 mins
Weight: 0 rows, 0.0 mins
lr_15: 0 rows, 0.0 mins
Mkt_lrt_15: 0 rows, 0.0 mins
Crypto_Index: 0 rows, 0.0 mins
slowK: 112 rows, 8.0 mins
slowD: 112 rows, 8.0 mins
fastK: 308 rows, 22.0 mins
fastD: 308 rows, 22.0 mins
rsi_60: 840 rows, 60.0 mins
macd_15_25: 392 rows, 28.0 mins
macd_signal_5: 392 rows, 28.0 mins
macd_hist: 392 rows, 28.0 mins
adx_40: 1106 rows, 79.0 mins
vol_sum_15: 196 rows, 14.0 mins
std_lr_15_5: 56 rows, 4.0 mins
std_Mkt_lrt_15_10: 126 rows, 9.0 mins
std_Crypto_Index_15: 196 rows, 14.0 mins
max nan mins=79.0


lr_15  Mkt_lrt_15  Crypto_Index      slowK      slowD  \
timestamp  Asset_ID                                                             
1641772800 13       -0.003493   -0.003263   7532.261935  65.391086  55.022644   
           10       -0.003462   -0.003263   7532.261935  27.791994  26.123390   
           9        -0.004696   -0.003263   7532.261935  29.197168  25.312999   
           8        -0.001899   -0.003263   7532.261935  70.696046  72.645704   
           6        -0.003110   -0.003263   7532.261935  26.109021  25.788997   
           7        -0.003969   -0.003263   7532.261935  28.621263  29.365428   
           1        -0.002324   -0.003263   7532.261935  46.712745  41.673744   
           4        -0.003500   -0.003263   7532.261935  42.674167  45.043987   
           0        -0.002281   -0.003263   7532.261935  50.543600  53.592032   
           2        -0.002764   -0.003263   7532.261935  63.298828  59.702026   
           3        -0.005190   -0.003263   7532.261935  19.232853  20.879361   
           12       -0.003568   -0.003263   7532.261935  27.164509  31.349793   
           5        -0.005577   -0.003263   7532.261935  39.841126  45.648873   
           11       -0.000454   -0.003263   7532.261935  72.523292  74.393164   

                         fastK      fastD     rsi_60  macd_15_25  \
timestamp  Asset_ID                                                
1641772800 13        22.058824  20.405045  43.458805   -0.000011   
           10        17.623554  60.294651  46.323024    1.481427   
           9         27.911018  21.723330  41.846081   -0.050992   
           8         65.358362  57.977789  49.931053    0.001525   
           6         23.688969  38.170847  37.944579   -1.164433   
           7         12.822458  18.629441  42.486766   -0.006396   
           1         34.741660  39.723022  39.217776   -9.133145   
           4         15.257143  30.317460  40.538579   -0.000031   
           0         21.961674  43.766773  42.279728   -0.004755   
           2         28.915663  32.137188  43.653350   -0.033553   
           3         12.150235  31.617458  39.320309   -0.000538   
           12        16.924959  37.240995  45.065683    0.000017   
           5         11.861472  27.906087  42.963708   -0.000998   
           11        56.107383  70.482249  49.642692    0.126525   

                     macd_signal_5     macd_hist     adx_40    vol_sum_15  \
timestamp  Asset_ID                                                         
1641772800 13            -0.000010 -3.124546e-07  17.208594  1.020347e+07   
           10             1.885207 -4.037797e-01  18.684020  2.962160e+01   
           9             -0.043601 -7.390640e-03  19.544074  5.065137e+03   
           8              0.001004  5.207067e-04  79.307572  5.350608e+04   
           6             -1.024059 -1.403742e-01  28.328100  5.335395e+03   
           7             -0.004225 -2.171462e-03  18.835485  5.021665e+03   
           1             -7.835613 -1.297532e+00  32.151451  5.309589e+02   
           4             -0.000022 -8.873135e-06  19.521657  4.875971e+06   
           0              0.011159 -1.591443e-02  22.395121  5.152490e+03   
           2             -0.026942 -6.610882e-03  24.349438  7.295325e+02   
           3             -0.000397 -1.403558e-04  23.837558  1.410050e+06   
           12             0.000042 -2.568999e-05  14.130360  1.559958e+06   
           5             -0.000703 -2.944162e-04  17.907176  1.716800e+05   
           11             0.130126 -3.600843e-03  54.173581  8.948188e+02   

                     std_lr_15_5  std_Mkt_lrt_15_10  std_Crypto_Index_15  
timestamp  Asset_ID                                                       
1641772800 13           0.000853           0.001532             3.874972  
           10           0.003268           0.001532             3.874972  
           9            0.001116           0.001532             3.874972  
           8            0.0134

In [92]:
pre_minute = 200
sup_train2 = sup_train.iloc[-14*pre_minute:,:]
sup_train2= sup_train2.groupby('Asset_ID').apply(lambda x: lag_features(x,**fdict))
sup_train2[sup_train2.isin([np.nan]).any(axis=1)]
sup_train2.iloc[-14:,8:]

lr_15  Mkt_lrt_15  Crypto_Index      slowK      slowD  \
timestamp  Asset_ID                                                             
1641772800 13       -0.003493   -0.003263   7532.261935  65.391086  55.022644   
           10       -0.003462   -0.003263   7532.261935  27.791994  26.123390   
           9        -0.004696   -0.003263   7532.261935  29.197168  25.312999   
           8        -0.001899   -0.003263   7532.261935  70.696046  72.645704   
           6        -0.003110   -0.003263   7532.261935  26.109021  25.788997   
           7        -0.003969   -0.003263   7532.261935  28.621263  29.365428   
           1        -0.002324   -0.003263   7532.261935  46.712745  41.673744   
           4        -0.003500   -0.003263   7532.261935  42.674167  45.043987   
           0        -0.002281   -0.003263   7532.261935  50.543600  53.592032   
           2        -0.002764   -0.003263   7532.261935  63.298828  59.702026   
           3        -0.005190   -0.003263   7532.261935  19.232853  20.879361   
           12       -0.003568   -0.003263   7532.261935  27.164509  31.349793   
           5        -0.005577   -0.003263   7532.261935  39.841126  45.648873   
           11       -0.000454   -0.003263   7532.261935  72.523292  74.393164   

                         fastK      fastD     rsi_60  macd_15_25  \
timestamp  Asset_ID                                                
1641772800 13        22.058824  20.405045  43.186819   -0.000011   
           10        17.623554  60.294651  46.081308    1.481427   
           9         27.911018  21.723330  41.847646   -0.050992   
           8         65.358362  57.977789  49.920025    0.001525   
           6         23.688969  38.170847  37.691425   -1.164433   
           7         12.822458  18.629441  42.248069   -0.006396   
           1         34.741660  39.723022  39.076006   -9.133139   
           4         15.257143  30.317460  40.315984   -0.000031   
           0         21.961674  43.766773  42.278914   -0.004755   
           2         28.915663  32.137188  43.872510   -0.033552   
           3         12.150235  31.617458  39.020115   -0.000538   
           12        16.924959  37.240995  45.050216    0.000017   
           5         11.861472  27.906087  42.925061   -0.000998   
           11        56.107383  70.482249  49.826329    0.126525   

                     macd_signal_5     macd_hist     adx_40    vol_sum_15  \
timestamp  Asset_ID                                                         
1641772800 13            -0.000010 -3.124574e-07  17.855194  1.020347e+07   
           10             1.885206 -4.037796e-01  19.749248  2.962160e+01   
           9             -0.043601 -7.390644e-03  19.565463  5.065137e+03   
           8              0.001004  5.207069e-04  80.001199  5.350608e+04   
           6             -1.024059 -1.403742e-01  28.996887  5.335395e+03   
           7             -0.004225 -2.171462e-03  18.838310  5.021665e+03   
           1             -7.835606 -1.297533e+00  32.056602  5.309589e+02   
           4             -0.000022 -8.873141e-06  20.532677  4.875971e+06   
           0              0.011159 -1.591446e-02  22.231004  5.152490e+03   
           2             -0.026942 -6.610892e-03  23.752491  7.295325e+02   
           3             -0.000397 -1.403559e-04  24.743019  1.410050e+06   
           12             0.000042 -2.569000e-05  14.161012  1.559958e+06   
           5             -0.000703 -2.944163e-04  18.368746  1.716800e+05   
           11             0.130126 -3.600849e-03  53.826693  8.948188e+02   

                     std_lr_15_5  std_Mkt_lrt_15_10  std_Crypto_Index_15  
timestamp  Asset_ID                                                       
1641772800 13           0.000853           0.001532             3.874972  
           10           0.003268           0.001532             3.874972  
           9            0.001116           0.001532             3.874972  
           8            0.0134

In [54]:
fdict.keys()

dict_keys(['std_lr_15', 'std_Mkt_lrt_15', 'std_Crypto_Index', 'rsi', 'macd_sig', 'macd_s', 'macd_l', 'lrtn', 'fastk2', 'fastk1', 'beta_s', 'beta_l', 'adx'])

In [53]:
##pre_minute make sure the last minute can get non-nan values
pre_minute = max([fdict['std_lr_15']+15,fdict['std_Mkt_lrt_15'],fdict['std_Crypto_Index'],
                  fdict['rsi']+1,fdict['macd_l']+5,fdict['lrtn']+1,
                  fdict['fastk2']+10,fdict['fastk1']+10,fdict['adx']*2+1])
pre_minute

81